In [ ]:
import numpy as np
import pandas as pd

from scipy.optimize import minimize_scalar

from pta_learn.tpmr import TPMR

from tsfresh import extract_features
from tsfresh.feature_extraction import MinimalFCParameters

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, mean_squared_error, mean_absolute_error, r2_score

from imblearn.over_sampling import RandomOverSampler

import pickle
import os
import ast

In [ ]:
def load_markup(markup_path):
    df = pd.read_csv(markup_path, sep=';')
    df['recovery'] = df['recovery'].apply(ast.literal_eval)
    df['drop'] = df['drop'].apply(ast.literal_eval)
    return df

In [ ]:
def preprocess_segment(df, grid_timestep=0.01, smoothing_time_window=5, interp_method='time'):
    seg = df.copy()
    seg.set_index('Time', inplace=True)
    stt, endt = np.floor(seg.index.min()), np.ceil(seg.index.max())
    uniform_times = np.arange(stt, endt + 1, grid_timestep)
    
    seg.index = pd.to_timedelta(seg.index, unit='h')
    new_times = pd.to_timedelta(uniform_times, unit='h')
    union_idx = seg.index.union(new_times)
    
    # Interpolate and fill remaining NaNs
    seg_interp = seg.reindex(union_idx).interpolate(interp_method).ffill().bfill()
    
    index_window_size = int(float(smoothing_time_window) / float(grid_timestep))
    index_window_size = int(index_window_size)  # Ensure it's an integer
    
    seg_uniform = seg_interp.reindex(new_times)
    
    # Apply rolling mean and fill NaNs resulting from the window
    seg_uniform = seg_uniform.rolling(window=index_window_size, center=True, min_periods=1).mean()
    
    # Handle any remaining edge NaNs (though min_periods=1 should prevent them)
    seg_uniform = seg_uniform.ffill().bfill()
    
    seg_uniform.index = seg_uniform.index.total_seconds() / 3600
    seg_uniform.reset_index(inplace=True)
    seg_uniform.columns = ['Time', 'Pressure']
    seg_uniform['Timestamp'] = pd.to_timedelta(seg_uniform['Time'], unit='h')
    return seg_uniform

In [ ]:
def load_series(series_path):
    df = pd.read_csv(series_path, delimiter='\t', header=None, names=['Time', 'Pressure'])
    return df

In [ ]:
def split_series_by_gap(df,gap):
    segments = []
    start_idx = 0
    ts = df['Time'].values
    for i in range(1, len(ts)):
        if (ts[i] - ts[i-1]) > gap:
            segments.append(df.iloc[start_idx:i].copy())
            start_idx = i
    if start_idx < len(ts):
        segments.append(df.iloc[start_idx:].copy())
    return segments

In [ ]:
def get_recovery_intervals(df_bhp, p = 1, int_len_treshhold = 70):
    df = df_bhp.copy()
    df['Pressure'] = -df['Pressure'].values + np.max(df['Pressure'].values)
    shutin_bp_all, shutin_bp_interval, shutin_transient_all, shutin_transient_interval = TPMR(df, p, int_len_treshhold)
    
    if shutin_transient_interval.empty:
        shutin_transient_interval = pd.DataFrame(columns = ['start','end'])
    else:
        shutin_transient_interval = shutin_transient_interval[['start/hr', 'end/hr']]
        shutin_transient_interval.columns = ['start','end']
    return shutin_transient_interval

In [ ]:
def get_drop_intervals(df_bhp, p = 1, int_len_treshhold = 70):
    shutin_bp_all, shutin_bp_interval, shutin_transient_all, shutin_transient_interval = TPMR(df_bhp, p, int_len_treshhold)
    if shutin_transient_interval.empty:
        shutin_transient_interval = pd.DataFrame(columns = ['start','end'])
    else:
        shutin_transient_interval = shutin_transient_interval[['start/hr', 'end/hr']]
        shutin_transient_interval.columns = ['start','end']
    return shutin_transient_interval

In [ ]:
def fit_ln_curve(df):
    df = df[df['Time'] > 0].copy()
    
    def objective(epsilon_shift):
        df['lnTime'] = np.log(df['Time'] + epsilon_shift)
        
        X = np.vstack([df['lnTime'], np.ones(len(df))]).T
        y = df['Pressure'].values
        
        coeff, _, _, _ = np.linalg.lstsq(X, y, rcond=None)
        q, b = coeff
        
        y_pred = q * df['lnTime'] + b
        
        return mean_squared_error(y, y_pred)
    
    result = minimize_scalar(objective, bounds=(-min(df['Time']), 1), method='bounded')
    best_epsilon_shift = result.x
    
    df['lnTime'] = np.log(df['Time'] + best_epsilon_shift)
    X = np.vstack([df['lnTime'], np.ones(len(df))]).T
    y = df['Pressure'].values
    coeff, _, _, _ = np.linalg.lstsq(X, y, rcond=None)
    q, b = coeff
    y_pred = q * df['lnTime'] + b
    
    mse = mean_squared_error(y, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y, y_pred)
    r2 = r2_score(y, y_pred)
    
    return {'q': q, 'b': b, 'MSE': mse, 'RMSE': rmse, 'MAE': mae, 'R2': r2, 'Epsilon Shift': best_epsilon_shift}

# Функции для трейна

In [ ]:
def overlap_ratio(interval1, interval2):
    start1, end1 = interval1
    start2, end2 = interval2
    intersection = max(0, min(end1, end2) - max(start1, start2))
    union = max(end1, end2) - min(start1, start2)
    if union == 0:
        return 0
    return intersection / union

In [ ]:
def max_overlap_from_df2(row, df2):
    start1, end1 = row['start'], row['end']
    overlaps = df2.apply(lambda x: overlap_ratio((start1, end1), (x['start'], x['end'])), axis=1)
    return overlaps.max()

In [ ]:
def filter_intervals(int_pred, int_true,overlap_threshold = 0.5):
    # Вычисляем для каждого интервала из df1 максимальное перекрытие с интервалами из df2
    int_pred['max_overlap'] = int_pred.apply(lambda row: max_overlap_from_df2(row, int_true), axis=1)
    # Фильтруем интервалы, максимальное перекрытие которых меньше порога
    filtered_df1 = int_pred[int_pred['max_overlap'] < overlap_threshold]
    return filtered_df1

In [ ]:
def compute_tsfresh_features(df):
    # For a single time series, add an id column (all rows belong to the same series)
    df['id'] = 1  
    
    # Extract features; the result is a DataFrame with one row per id
    features_df = extract_features(df, 
                                   column_id='id',
                                   column_sort='Time',
                                   column_value='Pressure',
                                   default_fc_parameters = MinimalFCParameters(),
                                   n_jobs = 1)
    
    # Convert the features of our single time series to a dict
    features_dict = features_df.iloc[0].to_dict()
    return features_dict

In [ ]:
def build_train_set(df_markup, dataset_path):
    dataset = []
    for index, row in df_markup.iterrows():
        print(f"№{index}/{len(df_markup)}: {row['file']}")
        series_path = os.path.join(dataset_path, row['file'])
        if not os.path.exists(series_path):
            print(f"path {series_path} does not exist")
            continue
            
        series = load_series(series_path)
        segments = split_series_by_gap(series, 50)
        
        recovery_ints = pd.DataFrame(row['recovery'], columns=['start', 'end'])
        drop_ints = pd.DataFrame(row['drop'], columns=['start', 'end'])
        
        recovery_ints_pred = []
        drop_ints_pred = []
        for seg in segments:
            if len(seg) < 20:
                continue
            proc_seg = preprocess_segment(seg, smoothing_time_window = 7)
            recovery_ints_pred.append(get_recovery_intervals(proc_seg))
            drop_ints_pred.append(get_drop_intervals(proc_seg))

        recovery_ints_pred = pd.concat(recovery_ints_pred)
        drop_ints_pred = pd.concat(drop_ints_pred)
        
        recovery_ints['type'] = 'recovery'
        drop_ints['type'] = 'drop'
        recovery_ints_pred['type'] = 'recovery'
        drop_ints_pred['type'] = 'drop'
        
        if recovery_ints.empty:
            filtered_recovery = recovery_ints.copy()
        else:
            filtered_recovery = filter_intervals(recovery_ints_pred, recovery_ints)

        if drop_ints.empty:
            filtered_drop = drop_ints.copy()
        else:
            filtered_drop = filter_intervals(drop_ints_pred, drop_ints)
            
        true_ints = pd.concat([recovery_ints, drop_ints])
        filtered_ints = pd.concat([filtered_recovery, filtered_drop])
        true_ints['class'] = 1
        filtered_ints['class'] = 0
        all_ints = pd.concat([true_ints, filtered_ints])

        print(f"{len(recovery_ints)} true recovery intervals")
        print(f"{len(drop_ints)} true drop intervals")
        print(f"{len(recovery_ints_pred)} predicted recovery intervals")
        print(f"{len(drop_ints_pred)} predicted drop intervals")
        print(f"{len(filtered_recovery)} filtered recovery intervals")
        print(f"{len(filtered_drop)} filtered drop intervals")
        print("|-------------------------------------------|")
        for _, interval in all_ints.iterrows():
            segment = series[(series['Time'] >= interval['start']) & (series['Time'] <= interval['end'])]
            if len(segment) < 20:
                continue
            seg = segment.copy()
            seg['Time'] = seg['Time'].values - np.min(seg['Time'].values)
            output = fit_ln_curve(seg)
            output['file'] = row['file']
            output['interval_start'] = interval['start']
            output['interval_end'] = interval['end']
            output['type'] = interval['type']
            output['class'] = interval['class']

            # features
            output['duration'] = interval['end'] - interval['start']
            output['len'] = len(seg)
            output['type_binary'] = 1 if interval['type'] == 'recovery' else 0
            output['amplitude'] = seg['Pressure'].max() - seg['Pressure'].min()

            tsfresh_feat = compute_tsfresh_features(seg)
            output.update(tsfresh_feat)
            dataset.append(output)
            
    return pd.DataFrame(dataset)

In [ ]:
def train(data):
    X = data.drop(columns=['class'])
    y = data['class']
    
    # Optional
    #scaler = StandardScaler()
    #X_scaled = scaler.fit_transform(X)
    X_scaled = X
    
    ros = RandomOverSampler(random_state=42)
    X_resampled, y_resampled = ros.fit_resample(X_scaled, y)
    
    X_train, X_test, y_train, y_test = train_test_split(
        X_resampled, y_resampled, test_size=0.3, random_state=42
    )
    
    model = RandomForestClassifier(n_estimators=50, random_state=42)
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)
    
    accuracy = accuracy_score(y_test, y_pred)
    conf_matrix = confusion_matrix(y_test, y_pred)
    class_report = classification_report(y_test, y_pred)
    
    print("Model Evaluation:")
    print("--------------------------------------")
    print("Accuracy:", accuracy)
    print("\nConfusion Matrix:\n", conf_matrix)
    print("\nClassification Report:\n", class_report)

    return model

# Функции для теста

In [ ]:
dataset_path = r'./dataset'
markup_csv_path = r'./markup.csv'
train_path = r'./train.csv'

In [ ]:
markup = load_markup(markup_csv_path)

In [ ]:
if os.path.exists(train_path):
    print("load existing training set")
    df_train = pd.read_csv(train_path)
else:
    df_train = build_train_set(markup, dataset_path)
    df_train.to_csv(train_path, index=False)

In [ ]:
train_feats = df_train.drop(['file','interval_start','interval_end','type'], axis = 1)
model = train(train_feats)

In [ ]:
with open('random_forest_model.pkl', 'wb') as file:
    pickle.dump(model, file)